In [1]:
# Import libraries/load environments

# Import required libraries
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
env_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/stockapi.env'  # Replace with your actual path
load_dotenv(env_path)
api_key = os.getenv('ALPHA_VANTAGE_API_KEY')

# Define Paths and Parameters

# Define root paths for sub-sectors
root_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24'

# Sub-sector paths (replace with actual paths later)
sub_sector_paths = {
    "Aerospace_Defense": f"{root_path}/AEROSPACE_DEFENSE",
    "Building_Products": f"{root_path}/BUILDING_PRODUCTS",
    "Construction_Engineering": f"{root_path}/CONSTRUCTION_ENGINEERING",
    "Electrical_Components_Equipment": f"{root_path}/ELECTRICAL_COMPONENTS_EQUIPMENT",
    "Heavy_Electrical_Equipment": f"{root_path}/HEAVY_ELECTRICAL_EQUIPMENT",
    "Industrial_Conglomerates": f"{root_path}/INDUSTRIAL_CONGLOMERATES",
    "Construction_Machinery_Equipment": f"{root_path}/CONSTRUCTION_MACHINERY_EQUIPMENT",
    "Agricultural_Farm_Machine": f"{root_path}/AGRICULTURAL_FARM_MACHINE",
    "Industrial_Machinery_Components": f"{root_path}/INDUSTRIAL_MACHINERY_COMPONENTS",
    "Trading_Companies_Distributors": f"{root_path}/TRADING_COMPANIES_DISTRIBUTORS",
    "Commercial_Printing": f"{root_path}/COMMERCIAL_PRINTING",
    "Environmental_Facilities_Services": f"{root_path}/ENVIRONMENTAL_FACILITIES_SERVICES",
    "Office_Services_Supplies": f"{root_path}/OFFICE_SERVICES_SUPPLIES",
    "Diversified_Support_Services": f"{root_path}/DIVERSIFIED_SUPPORT_SERVICES",
    "Security_Alarm_Services": f"{root_path}/SECURITY_ALARM_SERVICES",
    "Human_Resources_Employment_Services": f"{root_path}/HUMAN_RESOURCES_EMPLOYMENT_SERVICES",
    "Research_Consulting_Services": f"{root_path}/RESEARCH_CONSULTING_SERVICES",
    "Data_Processing_Outsourcing_Services": f"{root_path}/DATA_PROCESSING_OUTSOURCING_SERVICES",
    "Air_Freight_Logistics": f"{root_path}/AIR_FREIGHT_LOGISTICS",
    "Passenger_Airlines": f"{root_path}/PASSENGER_AIRLINES",
    "Marine_Transportation": f"{root_path}/MARINE_TRANSPORTATION",
    "Rail_Transportation": f"{root_path}/RAIL_TRANSPORTATION",
    "Cargo_Ground_Transportation": f"{root_path}/CARGO_GROUND_TRANSPORTATION",
    "Passenger_Ground_Transportation": f"{root_path}/PASSENGER_GROUND_TRANSPORTATION",
    "Airport_Services": f"{root_path}/AIRPORT_SERVICES",
    "Highways_Railtracks": f"{root_path}/HIGHWAYS_RAILTRACKS",
    "Marine_Ports_Services": f"{root_path}/MARINE_PORTS_SERVICES"
}

# Define years
years = ["2020", "2021", "2022", "2023", "2024"]

# Define sub-sector symbols (these are examples, please update with actual symbols)
sub_sector_symbols = {
    "Aerospace_Defense": ["BA", "LMT", "NOC"],  # Replace with actual symbols
    "Building_Products": ["MAS", "FBHS", "ALLE"],  # Replace with actual symbols
    "Construction_Engineering": ["FLR", "JEC", "ACM"],  # Replace with actual symbols
    "Electrical_Components_Equipment": ["EMR", "HON", "ROK"],  # Replace with actual symbols
    "Heavy_Electrical_Equipment": ["GE", "ETN", "ABB"],  # Replace with actual symbols
    "Industrial_Conglomerates": ["MMM", "HON", "GE"],  # Replace with actual symbols
    "Construction_Machinery_Equipment": ["CAT", "DE", "TEX"],  # Replace with actual symbols
    "Agricultural_Farm_Machine": ["AGCO", "CNHI", "DE"],  # Replace with actual symbols
    "Industrial_Machinery_Components": ["DOV", "IR", "SWK"],  # Replace with actual symbols
    "Trading_Companies_Distributors": ["FAST", "GWW", "AIT"],  # Replace with actual symbols
    "Commercial_Printing": ["DLX", "QUAD", "RRD"],  # Replace with actual symbols
    "Environmental_Facilities_Services": ["WM", "RSG", "CLH"],  # Replace with actual symbols
    "Office_Services_Supplies": ["SPLS", "ODP", "ACCO"],  # Replace with actual symbols
    "Diversified_Support_Services": ["ABM", "ROL", "VVI"],  # Replace with actual symbols
    "Security_Alarm_Services": ["ADT", "ALLE", "TYC"],  # Replace with actual symbols
    "Human_Resources_Employment_Services": ["MAN", "RHI", "KFY"],  # Replace with actual symbols
    "Research_Consulting_Services": ["IT", "HURN", "FCN"],  # Replace with actual symbols
    "Data_Processing_Outsourcing_Services": ["ADP", "PAYX", "VRSK"],  # Replace with actual symbols
    "Air_Freight_Logistics": ["FDX", "UPS", "XPO"],  # Replace with actual symbols
    "Passenger_Airlines": ["DAL", "AAL", "UAL"],  # Replace with actual symbols
    "Marine_Transportation": ["MATX", "KEX", "GNK"],  # Replace with actual symbols
    "Rail_Transportation": ["CSX", "NSC", "UNP"],  # Replace with actual symbols
    "Cargo_Ground_Transportation": ["JBHT", "ODFL", "SAIA"],  # Replace with actual symbols
    "Passenger_Ground_Transportation": ["LYFT", "UBER", "BLBD"],  # Replace with actual symbols
    "Airport_Services": ["ASR", "PAC", "OMAB"],  # Replace with actual symbols
    "Highways_Railtracks": ["CNI", "CP", "KSU"],  # Replace with actual symbols
    "Marine_Ports_Services": ["HMT", "SYD", "NWY"],  # Replace with actual symbols
}

# Define the Data Fetching and Saving Function

# Function to fetch data from Alpha Vantage and save as CSV
def fetch_and_save_data(symbol, sub_sector, year, api_key, output_dir):
    base_url = "https://www.alphavantage.co/query?"
    function = "TIME_SERIES_DAILY_ADJUSTED"
    
    params = {
        "function": function,
        "symbol": symbol,
        "outputsize": "full",
        "apikey": api_key,
        "datatype": "json"
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()

    if "Time Series (Daily)" in data:
        time_series = data["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient='index')
        df.index = pd.to_datetime(df.index)
        df.columns = [
            "Open", "High", "Low", "Close", "Adjusted Close", "Volume", 
            "Dividend Amount", "Split Coefficient"
        ]
        df = df.sort_index()

        # Filter data for the selected year
        df = df.loc[(df.index >= f"{year}-01-01") & (df.index <= f"{year}-12-31")]

        # Add an "Index" column
        df['Index'] = sub_sector

        # Save to CSV
        output_path = os.path.join(output_dir, f"{symbol}_{year}.csv")
        df.to_csv(output_path)
        print(f"Data for {symbol} ({sub_sector}, {year}) saved to {output_path}")
    else:
        print(f"Error fetching data for {symbol}: {data.get('Error Message', 'Unknown error')}")

# Iterate Over Sub-Sectors and Years

# Iterate over sub-sectors
for sub_sector, symbols in sub_sector_symbols.items():
    # Iterate over years
    for year in years:
        # Set the output directory for the current sub-sector and year
        output_dir = f"{sub_sector_paths[sub_sector]}/{year}"
        os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

        # Fetch and save data for each symbol in the current sub-sector and year
        for symbol in symbols:
            fetch_and_save_data(symbol, sub_sector, year, api_key, output_dir)


Data for BA (Aerospace_Defense, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AEROSPACE_DEFENSE/2020/BA_2020.csv
Data for LMT (Aerospace_Defense, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AEROSPACE_DEFENSE/2020/LMT_2020.csv
Data for NOC (Aerospace_Defense, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AEROSPACE_DEFENSE/2020/NOC_2020.csv
Data for BA (Aerospace_Defense, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AEROSPACE_DEFENSE/2021/BA_2021.csv
Data for LMT (Aerospace_Defense, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Grou

Data for ACM (Construction_Engineering, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/CONSTRUCTION_ENGINEERING/2022/ACM_2022.csv
Data for FLR (Construction_Engineering, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/CONSTRUCTION_ENGINEERING/2023/FLR_2023.csv
Data for JEC (Construction_Engineering, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/CONSTRUCTION_ENGINEERING/2023/JEC_2023.csv
Data for ACM (Construction_Engineering, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/CONSTRUCTION_ENGINEERING/2023/ACM_2023.csv
Data for FLR (Construction_Engineeri

Data for ETN (Heavy_Electrical_Equipment, 2024) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/HEAVY_ELECTRICAL_EQUIPMENT/2024/ETN_2024.csv
Error fetching data for ABB: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
Data for MMM (Industrial_Conglomerates, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/INDUSTRIAL_CONGLOMERATES/2020/MMM_2020.csv
Data for HON (Industrial_Conglomerates, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/INDUSTRIAL_CONGLOMERATES/2020/HON_2020.csv
Data for GE (Industrial_Conglomerates, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Proje

Data for DE (Agricultural_Farm_Machine, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AGRICULTURAL_FARM_MACHINE/2020/DE_2020.csv
Data for AGCO (Agricultural_Farm_Machine, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AGRICULTURAL_FARM_MACHINE/2021/AGCO_2021.csv
Error fetching data for CNHI: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
Data for DE (Agricultural_Farm_Machine, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AGRICULTURAL_FARM_MACHINE/2021/DE_2021.csv
Data for AGCO (Agricultural_Farm_Machine, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group P

Data for GWW (Trading_Companies_Distributors, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/TRADING_COMPANIES_DISTRIBUTORS/2022/GWW_2022.csv
Data for AIT (Trading_Companies_Distributors, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/TRADING_COMPANIES_DISTRIBUTORS/2022/AIT_2022.csv
Data for FAST (Trading_Companies_Distributors, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/TRADING_COMPANIES_DISTRIBUTORS/2023/FAST_2023.csv
Data for GWW (Trading_Companies_Distributors, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/TRADING_COMPANIES_DISTRIBUTORS/2023

...

Data for FCN (Research_Consulting_Services, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RESEARCH_CONSULTING_SERVICES/2021/FCN_2021.csv
Data for IT (Research_Consulting_Services, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RESEARCH_CONSULTING_SERVICES/2022/IT_2022.csv
Data for HURN (Research_Consulting_Services, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RESEARCH_CONSULTING_SERVICES/2022/HURN_2022.csv
Data for FCN (Research_Consulting_Services, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RESEARCH_CONSULTING_SERVICES/2022/FCN_2022.csv
Data

Data for XPO (Air_Freight_Logistics, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AIR_FREIGHT_LOGISTICS/2022/XPO_2022.csv
Data for FDX (Air_Freight_Logistics, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AIR_FREIGHT_LOGISTICS/2023/FDX_2023.csv
Data for UPS (Air_Freight_Logistics, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AIR_FREIGHT_LOGISTICS/2023/UPS_2023.csv
Data for XPO (Air_Freight_Logistics, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/AIR_FREIGHT_LOGISTICS/2023/XPO_2023.csv
Data for FDX (Air_Freight_Logistics, 2024) saved to /Users/r

Data for GNK (Marine_Transportation, 2024) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/MARINE_TRANSPORTATION/2024/GNK_2024.csv
Data for CSX (Rail_Transportation, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RAIL_TRANSPORTATION/2020/CSX_2020.csv
Data for NSC (Rail_Transportation, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RAIL_TRANSPORTATION/2020/NSC_2020.csv
Data for UNP (Rail_Transportation, 2020) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/RAIL_TRANSPORTATION/2020/UNP_2020.csv
Data for CSX (Rail_Transportation, 2021) saved to /Users/ronaldsheaks/De

Data for LYFT (Passenger_Ground_Transportation, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/PASSENGER_GROUND_TRANSPORTATION/2021/LYFT_2021.csv
Data for UBER (Passenger_Ground_Transportation, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/PASSENGER_GROUND_TRANSPORTATION/2021/UBER_2021.csv
Data for BLBD (Passenger_Ground_Transportation, 2021) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/PASSENGER_GROUND_TRANSPORTATION/2021/BLBD_2021.csv
Data for LYFT (Passenger_Ground_Transportation, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/PASSENGER_GROUND_TRANSP

Data for KSU (Highways_Railtracks, 2022) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/HIGHWAYS_RAILTRACKS/2022/KSU_2022.csv
Data for CNI (Highways_Railtracks, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/HIGHWAYS_RAILTRACKS/2023/CNI_2023.csv
Data for CP (Highways_Railtracks, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/HIGHWAYS_RAILTRACKS/2023/CP_2023.csv
Data for KSU (Highways_Railtracks, 2023) saved to /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/INDUSTRIAL_2020_24/HIGHWAYS_RAILTRACKS/2023/KSU_2023.csv
Data for CNI (Highways_Railtracks, 2024) saved to /Users/ronaldsheaks/Desktop/